<a href="https://colab.research.google.com/github/nirupamgpta/Assignments/blob/main/GenAIL4_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install SpeechRecognition pydub googletrans==4.0.0-rc1 gTTS
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [37]:
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
from IPython.display import Audio, display
import tempfile
import os
from pydub import AudioSegment

In [38]:
LANGUAGES = {
    "English": "en",
    "Hindi": "hi",
    "Spanish": "es",
    "French": "fr",
    "German": "de"
}

In [39]:
from IPython.display import Javascript
from base64 import b64decode
from google.colab.output import eval_js
from pydub import AudioSegment

def record_audio(seconds=5):
    display(Javascript("""
    async function record(sec){
        const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
        const recorder = new MediaRecorder(stream);
        recorder.start();
        await new Promise(resolve => setTimeout(resolve, sec*1000));
        recorder.stop();
        const data = await new Promise(resolve => recorder.ondataavailable = e => resolve(e.data));
        const buffer = await data.arrayBuffer();
        const base64 = btoa(String.fromCharCode(...new Uint8Array(buffer)));
        return base64;
    }
    """))

    audio_base64 = eval_js(f"record({seconds})")
    audio_bytes = b64decode(audio_base64)

    # Save as webm first
    webm_file_path = tempfile.mktemp(suffix=".webm")
    with open(webm_file_path, "wb") as f:
        f.write(audio_bytes)

    # Convert webm to wav with specific codec
    wav_file_path = tempfile.mktemp(suffix=".wav")
    audio = AudioSegment.from_file(webm_file_path, format="webm")
    audio.export(wav_file_path, format="wav", codec="pcm_s16le")

    os.remove(webm_file_path) # Clean up the webm file

    return wav_file_path

In [40]:
def speech_to_text(audio_path, lang_code):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
    return r.recognize_google(audio, language=lang_code)

In [41]:
def translate_text(text, src, dest):
    translator = Translator()
    return translator.translate(text, src=src, dest=dest).text

In [42]:
def text_to_speech(text, lang_code):
    tts = gTTS(text=text, lang=lang_code)
    file_path = tempfile.mktemp(suffix=".mp3")
    tts.save(file_path)
    display(Audio(file_path, autoplay=True))

In [43]:
def voice_translator_app(input_language, output_language, record_seconds=5):
    print(f"🎙️ Speak in {input_language}...")

    audio_file = record_audio(record_seconds)

    try:
        text = speech_to_text(audio_file, LANGUAGES[input_language])
        print("📝 You said:", text)

        translated = translate_text(
            text,
            LANGUAGES[input_language],
            LANGUAGES[output_language]
        )

        print(f"🌍 Translated ({output_language}):", translated)
        print("🔊 Speaking output...")
        text_to_speech(translated, LANGUAGES[output_language])

    except Exception as e:
        print("❌ Error:", e)

In [45]:
voice_translator_app(
    input_language="English",
    output_language="German",
    record_seconds=6
)

🎙️ Speak in English...


<IPython.core.display.Javascript object>

📝 You said: good morning
🌍 Translated (German): Guten Morgen
🔊 Speaking output...
